<a href="https://colab.research.google.com/github/preetamjumech/MLP/blob/main/Preetam_Saha_MLP_pytorch_16_08_2022.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.modules import Linear
from torch.optim import SGD
import numpy as np
import plotly.express as px 
from sklearn.datasets import make_circles,make_blobs
import pandas as pd
from torch.utils.data import DataLoader,TensorDataset #for batch supply

In [ ]:
#perceptrons cannot separate non-linear datapoints
data_1, class_1= make_blobs(n_samples = 1000,
                            n_features=2, 
                            random_state =123,
                            centers =2)

In [ ]:
data_1

array([[ 4.64753425, -5.27659405],
       [ 4.88521607, -5.15482689],
       [-5.32393762,  1.05187193],
       ...,
       [ 6.26144571, -3.93208906],
       [ 3.10900819, -4.89987679],
       [ 3.29876532, -5.67158841]])

In [ ]:
class_1

array([0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0,
       0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1,
       0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0,
       1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1,
       1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1,
       1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1,
       1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0,
       0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0,
       1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1,
       1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1,
       1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1,
       1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0,
       1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0,

In [ ]:
data_1_df = pd.DataFrame(np.c_[data_1,class_1],columns=['x','y','col']) #np.c_ concatenation column wise

In [ ]:
fig_1=px.scatter(data_frame = data_1_df,x='x',y='y',color="col")
fig_1.update_traces(marker = dict(size=5)) #size of the marker,scatter plot is a trace
fig_1.show()

In [ ]:
#perceptron can separate this linear separation
class Perceptron(nn.Module): #inherit the characterics from pytorch.nn.module
  def __init__(self):
    super(Perceptron,self).__init__()
    self.out = nn.Linear(in_features = 2, out_features = 2) #1=binary_cross_entropy, 2 = categorical_cross_entropy
  def forward(self,x):
    x = self.out(x)
    return x

model_1 = Perceptron()


In [ ]:
list(model_1.parameters()) # 6 parametrs, 2 i/p, 2 o/p, 2 for bias for o/p

[Parameter containing:
 tensor([[-0.6007,  0.1500],
         [-0.2653,  0.2449]], requires_grad=True), Parameter containing:
 tensor([0.5429, 0.1590], requires_grad=True)]

In [ ]:
train_loader = DataLoader(TensorDataset(torch.tensor(data_1),
                                        torch.tensor(class_1)),
                          batch_size=32,
                          shuffle=True) #dataloader has the capability to create batch size, for training, shuffle = True

In [ ]:
opt = SGD(params = model_1.parameters(),lr = 0.01)

In [ ]:
epochs = 500
model_1 = model_1.to('cuda') #cuda is talking baout GPU
for epoch in range(epochs):
  training_loss = 0.0
  for data,target in train_loader:
    data = data.float().to('cuda')
    target = target.to('cuda')
    
    opt.zero_grad() #before any optimization delete the previous optimization
    output = torch.softmax(model_1(data),dim=1) #dim=1 , value = columnwise 
    
    loss = F.cross_entropy(output,target)  #sigmoid means single, softmax = 2 #firstoutput,then target
    loss.backward()
    opt.step() #in a single step weights are going to get updated
    training_loss += loss.item()
  if (epoch+1)%50 ==0:
    print("Training_loss: {}".format(training_loss))


Training_loss: 10.074448138475418
Training_loss: 10.05023968219757
Training_loss: 10.042031049728394
Training_loss: 10.038088232278824
Training_loss: 10.035664349794388
Training_loss: 10.033590257167816
Training_loss: 10.0325568318367
Training_loss: 10.031530022621155
Training_loss: 10.03084060549736
Training_loss: 10.030149519443512


In [ ]:
model_pred = model_1(torch.tensor(data_1).float().to('cuda')) #gradient functionality still attached

In [ ]:
pred = model_pred.detach().cpu().numpy()

In [ ]:
pd.crosstab(pred.argmax(axis=1),class_1) #just linear activation since classes are linearly separable, no relu, sigmoid, no softmax,

col_0,0,1
row_0,,
0,500,0
1,0,500


In [ ]:
data_2,class_2 = make_circles(n_samples = 1000, 
                              noise = 0.05,
                              random_state = 123,
                              factor = 0.6)

In [ ]:
data_2_df = pd.DataFrame(np.c_[data_2,class_2],columns=['x','y','col'])

In [ ]:
fig_2=px.scatter(data_frame = data_2_df,x='x',y='y',color="col")
fig_2.update_traces(marker = dict(size=5)) #size of the marker,scatter plot is a trace
fig_2.show()

In [ ]:
#perceptron can separate this linear separation
class MLP(nn.Module): #inherit the characterics from pytorch.nn.module
  def __init__(self):
    super(MLP,self).__init__()
    self.h = nn.Linear(in_features = 2, out_features=3)
    self.out = nn.Linear(in_features=3,out_features = 1) #1=binary_cross_entropy, 2 = categorical_cross_entropy
  def forward(self,x):
    x = self.out(x)
    x = torch.sigmoid(x)
    return x

model_2 = MLP()

In [ ]:
train_loader = DataLoader(TensorDataset(torch.tensor(data_2),
                                        torch.tensor(class_2)),
                          batch_size=32,
                          shuffle=True) #dataloader has the capability to create batch size, for training, shuffle = True

In [ ]:
opt_2 = SGD(params = model_2.parameters(),lr = 0.01)

In [ ]:
epochs = 500
model_2 = model_2.to('cuda') #cuda is talking baout GPU
for epoch in range(epochs):
  training_loss = 0.0
  for data,target in train_loader:
    data = data.float().to('cuda')
    target = target.to('cuda')
    
    opt.zero_grad() #before any optimization delete the previous optimization
    output = torch.softmax(model_2(data),dim=1) #dim=1 , value = columnwise 
    
    loss = F.cross_entropy(output,target)  #sigmoid means single, softmax = 2 #firstoutput,then target
    loss.backward()
    opt.step() #in a single step weights are going to get updated
    training_loss += loss.item()
  if (epoch+1)%50 ==0:
    print("Training_loss: {}".format(training_loss))


RuntimeError: ignored